## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
import ipywidgets as widgets

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,82.45,78,75,17.27,broken clouds
1,1,Cape Town,ZA,-33.9258,18.4232,68.34,69,78,4.03,broken clouds
2,2,Thompson,CA,55.7435,-97.8558,-14.78,97,100,3.33,overcast clouds
3,3,Katsuura,JP,35.1333,140.3000,41.14,39,0,11.10,clear sky
4,4,Hobart,AU,-42.8794,147.3294,73.06,50,20,1.01,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
25,25,Mahebourg,MU,-20.4081,57.7000,79.25,80,100,16.87,overcast clouds
32,32,Rikitea,PF,-23.1203,-134.9692,79.57,69,100,13.18,overcast clouds
36,36,Bambous Virieux,MU,-20.3428,57.7575,79.21,80,100,16.51,overcast clouds
37,37,Puerto Ayora,EC,-0.7393,-90.3518,75.54,92,29,4.00,scattered clouds
45,45,Mackay,AU,-21.1500,149.2000,79.68,71,78,20.38,broken clouds
50,50,Vaini,TO,-21.2000,-175.2000,78.96,89,100,16.11,light rain
52,52,Nong Chik,TH,6.8436,101.1780,76.91,85,93,9.66,overcast clouds
60,60,Cap Malheureux,MU,-19.9842,57.6142,79.47,76,100,7.00,overcast clouds
66,66,Grand Gaube,MU,-20.0064,57.6608,79.38,76,100,7.00,overcast clouds
73,73,Mayumba,GA,-3.4320,10.6554,76.86,86,98,4.07,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Mahebourg,MU,79.25,overcast clouds,-20.4081,57.7000,
32,Rikitea,PF,79.57,overcast clouds,-23.1203,-134.9692,
36,Bambous Virieux,MU,79.21,overcast clouds,-20.3428,57.7575,
37,Puerto Ayora,EC,75.54,scattered clouds,-0.7393,-90.3518,
45,Mackay,AU,79.68,broken clouds,-21.1500,149.2000,
50,Vaini,TO,78.96,light rain,-21.2000,-175.2000,
52,Nong Chik,TH,76.91,overcast clouds,6.8436,101.1780,
60,Cap Malheureux,MU,79.47,overcast clouds,-19.9842,57.6142,
66,Grand Gaube,MU,79.38,overcast clouds,-20.0064,57.6608,
73,Mayumba,GA,76.86,overcast clouds,-3.4320,10.6554,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
25,Mahebourg,MU,79.25,overcast clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
32,Rikitea,PF,79.57,overcast clouds,-23.1203,-134.9692,People ThankYou
36,Bambous Virieux,MU,79.21,overcast clouds,-20.3428,57.7575,Casa Tia Villa
37,Puerto Ayora,EC,75.54,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
45,Mackay,AU,79.68,broken clouds,-21.1500,149.2000,International Lodge Motel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat","Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

#Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
from matplotlib import pyplot as plt

In [15]:
plt.savefig('WeatherPy_vacation_map.png')

<Figure size 432x288 with 0 Axes>